In [ ]:
from toolkits.datasets import load_next_30min
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import numpy as np
import torch
import torch.nn as nn
import os

In [2]:
class CNNModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cnnLayer = nn.Sequential(
            nn.Conv2d(in_channels=5, out_channels=16, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2,2), stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1, 1, 0),
        )

        self.fcLayer = nn.Sequential(
            nn.Linear(32 * 1 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 12),
            nn.Sigmoid(),
        )

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)

    def forward(self, x) -> torch.Tensor:
        x = self.cnnLayer(x)
        x = x.flatten(1)
        x = self.fcLayer(x)
        return x

    def save(self, ckpt_dir: str) -> None:
        torch.save(self.state_dict(), os.path.join(ckpt_dir, 'model.pth'))

    def load(self, ckpt_dir: str) -> None:
        self.load_state_dict(torch.load(os.path.join(ckpt_dir, 'model.pth')))

In [5]:
_, testDataset = load_next_30min()
testLoader = DataLoader(testDataset, batch_size=1024, shuffle=False)
model = CNNModel()
model.load('./model/')

In [7]:
model.eval()
speed_records = []
volume_records = []

for batch in tqdm(testLoader):
    X, y = batch
    with torch.no_grad():
        logits = model(X.to(model.device))
    label_speeds = np.array(y)[:,0] * 100
    label_volumes = np.array(y)[:,1] * 250
    pred_speeds = np.array(logits)[:,0] * 100
    pred_volumes = np.array(logits)[:,1] * 250

    # filt valid indices
    indices_speeds = np.where(label_speeds!=0)[0]
    indices_volumes = np.where(label_volumes!=0)[0]

    label_speeds = label_speeds[indices_speeds]
    label_volumes = label_volumes[indices_volumes]
    pred_speeds = pred_speeds[indices_speeds]
    pred_volumes = pred_volumes[indices_volumes]

    speed_records.append((np.abs(pred_speeds - label_speeds) / label_speeds).mean())
    volume_records.append((np.abs(pred_volumes - label_volumes) / label_volumes).mean())

100%|██████████| 1684/1684 [05:14<00:00,  5.36it/s]


In [8]:
print(f"MAPE (speed): {np.array(speed_records).mean():.2%}")
print(f"MAPE (volume): {np.array(volume_records).mean():.2%}")

MAPE (speed): 6.93%
MAPE (volume): 7.86%
